___

# [ Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## Python-gdal rasters


This notebook introduces how to work with raster data in Python using gdal binding. It covers: 

* 1. Reading raster data into Numpy ndarray

* 2. RGB image visualization

* 3. Simple map algebra 

* 4. Writing raster data into raster data file; 


### Setup

In [ ]:
# Common imports for reading, visualizing
import os
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 

# import gdal
from osgeo import gdal 

In [ ]:
import osgeo.gdal
print(osgeo.gdal.__version__)

In [ ]:
os.getcwd()

### Get data 

In [ ]:
# Set your own PATH!!! 
PATH = './data/'

if os.path.isdir(PATH): 
    print('ok')

In [ ]:
raster_fn = os.path.join(PATH, 'landsat.tif')

In [ ]:
if os.path.isfile(raster_fn): 
    print('ok')
else: 
    print('Check your path to the raster file!')

### Reading raster

In [ ]:
# open data source 
ds = gdal.Open(raster_fn) 
if ds is None: 
    print('Could not open {}'.format(fn)) 
    sys.exit(1) 

In [ ]:
ds

In [ ]:
# check raster dimensions 
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount

print('The ../{} raster dimensionos are: '.format(os.path.basename(raster_fn)))
print('{} cols, {} rows and {} bands'.format(cols, rows, bands))

In [ ]:
# geotransformation metadata
geotransform = ds.GetGeoTransform()

In [ ]:
print(geotransform)

In [ ]:
type(geotransform)

In [ ]:
# beaware the order of the metadata in the tuple
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]

print('The ./{} raster georeference info is: '.format(os.path.basename(raster_fn)))
print('origin X: {}, origin Y: {} and pixel size is: {}'.format(originX, originY, pixelWidth))

In [ ]:
# get value of pixel x, y from band 1 given spatial coorinates 
# beware we are already working with numpy array, while we know the origin coordiantes 
x1 = 631608
y1 = 7744535

xOffset = int((x1 - originX) / pixelWidth)
yOffset = int((y1 - originY) / pixelHeight)

# get band from datasource
band = ds.GetRasterBand(1)

# get data from band
# check API: https://gdal.org/api/python/osgeo.gdal.html 
# ReadAsArray(xoff=0, yoff=0, win_xsize=None, win_ysize=None, buf_xsize=None, buf_ysize=None, buf_type=None, buf_obj=None, resample_alg=0, callback=None, callback_data=None)
data1 = band.ReadAsArray(xOffset, yOffset, 1, 1)
print('Pixal value for x {} and y {} is {}'.format(x1, y1, data1[0]))

In [ ]:
# vector of spectral values across bands
data1_vector = []
for b in range(ds.RasterCount): 
    band = ds.GetRasterBand(b + 1)
    data1_vector.append(band.ReadAsArray(xOffset, yOffset, 1, 1)[0][0] / 10000)

print(data1_vector)

In [ ]:
# value of pixel x, y 
x2 = 635150
y2 = 7739676

xOffset2 = int((x2 - originX) / pixelWidth)
yOffset2 = int((y2 - originY) / pixelHeight)

# vector of spectral values 
data2_vector = []
for b in range(ds.RasterCount): 
    band = ds.GetRasterBand(b + 1)
    data2_vector.append(band.ReadAsArray(xOffset2, yOffset2, 1, 1)[0][0] / 10000)

print(data2_vector)

In [ ]:
landsat_wavelength = [485, 560, 660, 860, 1650, 2220]

plt.plot(landsat_wavelength, data1_vector, 'bo--')
plt.plot(landsat_wavelength, data2_vector, 'go--')

In [ ]:
# read full image 
data = band.ReadAsArray()
print('The image size is {}'.format(data.shape))

In [ ]:
# clean memory 
band = None
ds = None

### Writing raster

In [ ]:
# read image data bands to Numpy
driver = gdal.GetDriverByName('Gtiff')
ds = gdal.Open(raster_fn) 

if ds is None: 
    print('Could not open {}'.format(fn)) 
    sys.exit(1) 


img = np.zeros((ds.RasterYSize, ds.RasterXSize, ds.RasterCount), dtype=np.float64) 

for b in range(ds.RasterCount): 
    img[:, :, b] = ds.GetRasterBand(b + 1).ReadAsArray() / 10000


In [ ]:
img.shape

In [ ]:
# get meta
projection = ds.GetProjection()
geo_transform = ds.GetGeoTransform()

In [ ]:
img.shape

In [ ]:
plt.imshow(img[:,:,3], cmap='gray')
plt.colorbar()

In [ ]:
# view raster RGB using matplotlib 
index = np.array([4, 3, 2])
colors = img[:, :, index].astype(np.float64)
max_val = 0.4
min_val = 0.0

# enforce maximum and minimum values
colors[colors[:, :, :] > max_val] = max_val
colors[colors[:, :, :] < min_val] = min_val

for b in range(colors.shape[2]):
    colors[:, :, b] = colors[:, :, b] * 1 / (max_val - min_val)

plt.imshow(colors)

In [ ]:
# calculate snow (ice) index 
# NDSI = (GREEN - SWIR1) / (GREEN + SWIR1)
GREEN_ix = 1
SWIR1_ix = 4
NDSI = (img[:,:,GREEN_ix] - img[:,:,SWIR1_ix]) - (img[:,:,GREEN_ix] + img[:,:,SWIR1_ix])

NDSI.shape

In [ ]:
NDSI

In [ ]:
plt.imshow(NDSI, interpolation='none', cmap='RdBu', vmin=-0.3, vmax=0.05)
plt.colorbar()


In [ ]:
# save the new image
filename = os.path.join(PATH, 'landsat_ndsi.tif')
outDataset = driver.Create(filename, ds.RasterYSize, ds.RasterXSize, 1, gdal.GDT_Float32)

In [ ]:
# band to save 
# outBand = outDataset.GetRasterBand(1)
# outBand.WriteArray(NDSI)

outBand = outDataset.GetRasterBand(1).WriteArray(NDSI)


In [ ]:
# projection

In [ ]:
# set projection to datasource 
outDataset.SetProjection(projection)


In [ ]:
# set geotransform 
outDataset.SetGeoTransform(geo_transform)

In [ ]:
outDataset = None
# is it now saved? 


In [ ]:
os.path.isfile(filename)
